# Prédiction de la demande pour le modèle d'optimisation
Lors de la séance 8, nous avons créé et entraîné un modèle prédictif de la demande par UPC. Maintenant, dans ce notebook, nous allons préparer nos entrées pour le modèle d'optimisation en prédisant la demande pour différents niveaux de prix.

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Données d'entrée
Nous avons préparé les fichiers d'entrée qui contiennent les caractéristiques qui nous serviront à faire les prédictions. Le premier fichier contient un petit jeu de données tandis que le second fichier contient un plus gros jeu de données. En particulier,
1.   **'InputFeatures_Prob1.csv'**: C'est un problème à petite échelle. Les prédictions résultantes de ces données seront utilisées dans les modèles d'optimisation détaillé et compact.
2.   **'InputFeatures_Prob2.csv'**: C'est un problème à grande échelle. Les prédictions résultantes de ces données seront utilisées seulement dans le modèle compact.

*Pour faire les prédictions sur un jeu de données ou l'autre, vous devez définir `SIZE = 'small'` ou `SIZE = 'large'`.*

In [2]:
# Le paramètre SIZE doit être défini à 'small' ou 'large'
SIZE = 'small'

if SIZE == 'small':
    # petit exemple
    url = 'https://raw.githubusercontent.com/acedesci/scanalytics/master/data/InputFeatures_Prob1.csv'
elif SIZE == 'large':
    # grand exemple
    url = 'https://raw.githubusercontent.com/acedesci/scanalytics/master/data/InputFeatures_Prob2.csv'
else:
    raise ValueError("SIZE doit être défini à 'small' ou 'large'")

predDemand = pd.read_csv(url)
print(predDemand.shape)
predDemand

(12, 8)


,AVG_PRICE_VALUE,UPC,PRICE,PRICE_p2,FEATURE,DISPLAY,TPR_ONLY,RELPRICE
0,3.0,1600027528,2.5,6.25,0,0,0,0.833333
1,3.0,1600027528,3.0,9.00,0,0,0,1.000000
2,3.0,1600027528,3.5,12.25,0,0,0,1.166667
3,3.0,1600027564,2.5,6.25,0,0,0,0.833333
4,3.0,1600027564,3.0,9.00,0,0,0,1.000000
5,3.0,1600027564,3.5,12.25,0,0,0,1.166667
6,3.0,3000006340,2.5,6.25,0,0,0,0.833333
7,3.0,3000006340,3.0,9.00,0,0,0,1.000000
8,3.0,3000006340,3.5,12.25,0,0,0,1.166667
9,3.0,3800031829,2.5,6.25,0,0,0,0.833333


# Récupération de modèle
Ensuite, nous récupérons le meilleur modèle que nous avons précédemment entraîné et sauvegardé dans le répertoire de travail actuel (cwd) en fonction de l'une des deux options ci-dessous.

**Option 1 Jupyter**: si votre modèle est sauvegardé sur PC, vous devez indiquer le chemin d'accès aux modèles sauvegardés. Nous supposons ici qu'il se trouve dans le même dossier que le notebook.

In [3]:
cwd = './'

**Option 2 Google Drive (Colab)**: si votre modèle est enregistré sur Google Drive, vous pouvez accéder au dossier en le montant et en autorisant l'accès.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
cwd = '/content/drive/My Drive/'

En suivant le bloc ci-dessus, nous pouvons maintenant charger le modèle que nous avons précédemment formé et sauvegardé pour chaque UPC.

In [4]:
import pickle

productList = predDemand['UPC'].unique()

regr = {}
for upc in productList:
    filename = cwd+str(upc)+'_demand_model.sav'
    # load the model to disk
    regr[upc] = pickle.load(open(filename, 'rb'))

# Prévision de la demande
Dans cette cellule, nous exécutons également une boucle **pour** chaque UPC. Voici la descriptions de chaque ligne de la boucle:
* La première ligne de la boucle identifie les lignes dans `predDemand` qui correspondent à `upc`.
* La deuxième ligne récupère les caractéristiques de `upc`.
* La troisième ligne prend le modèle pour l'UPC courant (`regr[upc]`) et prédit la demande. Nous utilisons également la fonction `clip(min=0.)` pour s'assurer que la demande n'est pas négative et la fonction `round(decimals=1)` pour arrondir la valeur prédite à une décimale.
* La quatrième ligne place la demande prédite dans la colonne `'PRED_SALES'` de `predDemand`.

In [9]:
feature_list = ['PRICE', 'PRICE_p2', 'FEATURE', 'DISPLAY','TPR_ONLY','RELPRICE']

productList = predDemand['UPC'].unique()  # UPC pour predDemand
predDemand['PRED_SALES'] = -1.  # création de la colonne PRED_SALES
for upc in productList:
    mask = predDemand['UPC'] == upc  # lignes pour upc
    X = predDemand.loc[mask, feature_list].values  # caractéristiques pour upc
    y_pred = regr[upc].predict(X).clip(min=0).round(decimals=1)  # prédictions pour upc
    predDemand.loc[mask, 'PRED_SALES'] = y_pred  # ajout des prédictions aux données

predDemand

,AVG_PRICE_VALUE,UPC,PRICE,PRICE_p2,FEATURE,DISPLAY,TPR_ONLY,RELPRICE,PRED_SALES
0,3.0,1600027528,2.5,6.25,0,0,0,0.833333,94.9
1,3.0,1600027528,3.0,9.00,0,0,0,1.000000,67.0
2,3.0,1600027528,3.5,12.25,0,0,0,1.166667,46.4
3,3.0,1600027564,2.5,6.25,0,0,0,0.833333,24.1
4,3.0,1600027564,3.0,9.00,0,0,0,1.000000,22.6
5,3.0,1600027564,3.5,12.25,0,0,0,1.166667,19.8
6,3.0,3000006340,2.5,6.25,0,0,0,0.833333,6.2
7,3.0,3000006340,3.0,9.00,0,0,0,1.000000,4.0
8,3.0,3000006340,3.5,12.25,0,0,0,1.166667,3.0
9,3.0,3800031829,2.5,6.25,0,0,0,0.833333,32.9


Nous enregistrons maintenant les ventes prévues dans un fichier csv qui sera utilisé ultérieurement dans les modèles d'optimisation.

In [10]:
# Enregistrement des prédictions dans le fichier correspondant
if SIZE == 'small':
    predDemand.to_csv('predictedSales_Prob1.csv', index=False)
elif SIZE == 'large':
    predDemand.to_csv('predictedSales_Prob2.csv', index=False)
else:
    raise ValueError("SIZE doit être défini à 'small' ou 'large'")